<a href="https://colab.research.google.com/github/daisysong76/AI--Machine--learning/blob/main/Task_Aware_Distillation_and_Pruning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task-Aware Distillation and Pruning with a Transformer Model (BERT)
Here, we'll use Hugging Face’s DistilBERT as an example of model distillation and torch.nn.utils.prune for pruning. Suppose we're distilling and pruning a BERT model for a text classification task on a custom dataset.

1. Distillation: Train a DistilBERT model on a task-specific dataset using knowledge distillation from a pre-trained BERT model.
2. Pruning: Prune the smaller DistilBERT model for further efficiency.
Step 1: Set Up Libraries

In [1]:
!pip install torch transformers dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.54 which is incompatible.


In [ ]:
import torch
from transformers import BertForSequenceClassification, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

# Load a sample dataset for text classification
dataset = load_dataset("imdb")


Load the Teacher Model (BERT) and Student Model (DistilBERT)

In [ ]:
# Load pre-trained BERT as the teacher model
teacher_model = BertForSequenceClassification.from_pretrained("bert-base-uncased")

# Load DistilBERT as the student model
student_model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")


Define Distillation Training Loop

Define the training arguments and set up the Trainer for knowledge distillation. The student model will learn by matching its logits with the teacher's logits.

In [ ]:
from torch.nn import functional as F

# Define a custom distillation loss function
def distillation_loss(student_outputs, teacher_outputs, labels, alpha=0.5, temperature=2.0):
    soft_teacher_logits = F.log_softmax(teacher_outputs.logits / temperature, dim=-1)
    soft_student_logits = F.log_softmax(student_outputs.logits / temperature, dim=-1)

    # Cross-entropy between soft labels
    distill_loss = F.kl_div(soft_student_logits, soft_teacher_logits, reduction="batchmean") * (temperature ** 2)

    # Cross-entropy with ground truth labels
    hard_loss = F.cross_entropy(student_outputs.logits, labels)

    # Combine losses with weighting factor alpha
    return alpha * distill_loss + (1 - alpha) * hard_loss


Define a custom Trainer class to use the distillation loss:

In [ ]:
from transformers import TrainerCallback, TrainerState, TrainerControl

class DistillationTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)

        # Get teacher model predictions for distillation
        with torch.no_grad():
            teacher_outputs = teacher_model(**inputs)

        # Calculate the distillation loss
        loss = distillation_loss(outputs, teacher_outputs, labels)
        return (loss, outputs) if return_outputs else loss


Fine-Tune the Distilled Model

In [ ]:
training_args = TrainingArguments(
    output_dir="./distilled_model",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
)

trainer = DistillationTrainer(
    model=student_model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"]
)

# Start distillation training
trainer.train()


Prune the Fine-Tuned Model

Once the student model has been fine-tuned, we’ll apply pruning. Here, we’ll prune the linear layers in DistilBERT by removing low-magnitude weights.

In [ ]:
import torch.nn.utils.prune as prune

# Function to apply pruning to all linear layers
def apply_pruning(model, pruning_amount=0.3):
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Linear):
            prune.l1_unstructured(module, name="weight", amount=pruning_amount)
            prune.remove(module, 'weight')  # Make pruning permanent

# Apply pruning to the student model
apply_pruning(student_model, pruning_amount=0.3)


Save and Evaluate the Pruned Model
After pruning, save and evaluate the model on the test set to ensure it performs well despite the reduced size.

In [ ]:
# Save the pruned model
student_model.save_pretrained("./pruned_distilled_model")

# Evaluate the pruned model
eval_results = trainer.evaluate()
print(f"Pruned Model Evaluation Results: {eval_results}")
